# Timing Diagrams in Digital Design

*Understanding signal behavior over time*

---

## Prerequisites

This tutorial assumes familiarity with:

- **Binary numbers** and logic levels (HIGH/LOW, 1/0)
- **Logic gates** (AND, OR, NOT) and their behavior
- **Flip-flops** — basic understanding of sequential elements
- **Clock signals** — what they are and why they matter

---

## What is a Timing Diagram?

A timing diagram is a waveform chart that shows how digital signals change over time. It's the primary tool for:

- **Visualizing** circuit behavior across clock cycles
- **Debugging** timing-related bugs
- **Verifying** that signals meet setup/hold requirements
- **Communicating** design intent and specifications

Think of it as a "movie" of your circuit's signals, where each frame shows the logic level of every signal at that instant.

---

## Anatomy of a Timing Diagram

Let's start with a simple timing diagram showing a clock and a data signal:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 1, figsize=(12, 4), sharex=True)
fig.suptitle('Basic Timing Diagram Components', fontsize=14, fontweight='bold')

t = np.arange(0, 8, 0.01)

# Clock signal
clk = np.zeros_like(t)
for i in range(8):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Data signal
data_values = [0, 0, 1, 1, 1, 0, 1, 0]
data = np.zeros_like(t)
for i, val in enumerate(data_values):
    data[(t >= i) & (t < i + 1)] = val

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.3, 1.5)
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['0', '1'])
axes[0].grid(True, alpha=0.3, axis='x')

# Annotate clock features
axes[0].annotate('Rising edge', xy=(1.0, 0.5), xytext=(1.2, 1.3),
                fontsize=9, color='#1e40af',
                arrowprops=dict(arrowstyle='->', color='#1e40af', lw=1.5))
axes[0].annotate('Falling edge', xy=(1.5, 0.5), xytext=(1.8, -0.1),
                fontsize=9, color='#1e40af',
                arrowprops=dict(arrowstyle='->', color='#1e40af', lw=1.5))
axes[0].annotate('', xy=(3, 1.35), xytext=(4, 1.35),
                arrowprops=dict(arrowstyle='<->', color='#059669', lw=1.5))
axes[0].text(3.5, 1.45, 'Period', ha='center', fontsize=9, color='#059669')

# Plot data
axes[1].fill_between(t, 0, data, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, data, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('DATA', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.3, 1.5)
axes[1].set_yticks([0, 1])
axes[1].set_yticklabels(['0', '1'])
axes[1].set_xlabel('Time', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='x')

# Mark clock edges
for i in range(8):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

**Key elements:**

| Element | Description |
|---------|-------------|
| **Signal name** | Label on Y-axis (CLK, DATA, etc.) |
| **Logic levels** | HIGH (1) and LOW (0) positions |
| **Time axis** | Horizontal axis showing progression |
| **Rising edge** | Transition from LOW to HIGH (↑) |
| **Falling edge** | Transition from LOW to HIGH (↓) |
| **Period** | Time for one complete clock cycle |

---

## Signal Types in Timing Diagrams

Digital systems use several types of signals, each with distinct characteristics:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(5, 1, figsize=(12, 10), sharex=True)
fig.suptitle('Common Signal Types', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# 1. Clock signal - periodic
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])
axes[0].text(10.2, 0.5, 'Clock\n(periodic)', fontsize=9, va='center', color='#3b82f6')

# 2. Reset signal - active low, asynchronous
reset = np.ones_like(t)
reset[(t >= 0.3) & (t < 1.8)] = 0

axes[1].fill_between(t, 0, reset, color='#ef4444', alpha=0.3, step='pre')
axes[1].plot(t, reset, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('RST_N', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])
axes[1].text(10.2, 0.5, 'Reset\n(active-low)', fontsize=9, va='center', color='#ef4444')
axes[1].annotate('Active', xy=(1, 0.2), fontsize=8, color='#ef4444', ha='center')

# 3. Enable signal - control
enable = np.zeros_like(t)
enable[(t >= 2) & (t < 7)] = 1

axes[2].fill_between(t, 0, enable, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, enable, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('EN', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].text(10.2, 0.5, 'Enable\n(control)', fontsize=9, va='center', color='#f59e0b')

# 4. Data signal - changes on clock edges
data_values = [0, 0, 0, 1, 0, 1, 1, 0, 1, 0]
data = np.zeros_like(t)
for i, val in enumerate(data_values):
    data[(t >= i) & (t < i + 1)] = val

axes[3].fill_between(t, 0, data, color='#10b981', alpha=0.3, step='pre')
axes[3].plot(t, data, color='#10b981', lw=2, drawstyle='steps-pre')
axes[3].set_ylabel('D', fontsize=10, fontweight='bold')
axes[3].set_ylim(-0.2, 1.4)
axes[3].set_yticks([0, 1])
axes[3].text(10.2, 0.5, 'Data\n(synchronous)', fontsize=9, va='center', color='#10b981')

# 5. Bus signal - multi-bit represented as a group
bus_values = ['XX', '00', '00', '01', '02', '03', '03', '03', '04', '00']
bus = np.zeros_like(t)
for i in range(10):
    bus[(t >= i) & (t < i + 1)] = 0.5

axes[4].fill_between(t, 0.2, 0.8, color='#8b5cf6', alpha=0.2, step='pre')
axes[4].plot(t, np.ones_like(t) * 0.8, color='#8b5cf6', lw=2, drawstyle='steps-pre')
axes[4].plot(t, np.ones_like(t) * 0.2, color='#8b5cf6', lw=2, drawstyle='steps-pre')
# Draw transitions
for i in range(1, 10):
    if bus_values[i] != bus_values[i-1]:
        axes[4].plot([i, i], [0.2, 0.8], color='#8b5cf6', lw=2)
        # X crossing for transition
        axes[4].plot([i-0.05, i+0.05], [0.2, 0.8], color='#8b5cf6', lw=1)
        axes[4].plot([i-0.05, i+0.05], [0.8, 0.2], color='#8b5cf6', lw=1)
for i, val in enumerate(bus_values):
    axes[4].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color='#5b21b6')

axes[4].set_ylabel('BUS[7:0]', fontsize=10, fontweight='bold')
axes[4].set_ylim(-0.1, 1.1)
axes[4].set_yticks([])
axes[4].set_xlabel('Clock Cycles', fontsize=11)
axes[4].text(10.2, 0.5, 'Bus\n(multi-bit)', fontsize=9, va='center', color='#8b5cf6')

# Mark clock edges
for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

**Signal type characteristics:**

| Type | Characteristics | Examples |
|------|-----------------|----------|
| **Clock** | Periodic, fixed frequency | CLK, SYSCLK |
| **Reset** | Often active-low, assertion clears state | RST_N, RESET |
| **Enable** | Gates operations on/off | EN, CE, OE |
| **Data** | Changes relative to clock | D, Q, DATA_IN |
| **Bus** | Multi-bit values shown as hex/decimal | ADDR[15:0], DATA[7:0] |

---

## Clock Edges and Synchronous Design

In synchronous digital design, all state changes happen at **clock edges**. The two types are:

- **Rising edge (positive edge)**: LOW → HIGH transition
- **Falling edge (negative edge)**: HIGH → LOW transition

Most designs use **rising-edge triggered** flip-flops.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(12, 6), sharex=True)
fig.suptitle('D Flip-Flop: Input Captured on Rising Edge', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

# D input - changes between clock edges
d_values = [0, 1, 1, 0, 1, 0, 0, 1, 1, 0]
d_signal = np.zeros_like(t)
# D changes slightly after falling edge (mid-cycle)
for i, val in enumerate(d_values):
    d_signal[(t >= i + 0.6) & (t < i + 1.6)] = val
d_signal[(t >= 0) & (t < 0.6)] = 0

# Q output - captures D on rising edge
q_values = [0, 0, 1, 1, 0, 1, 0, 0, 1, 1]
q_signal = np.zeros_like(t)
for i, val in enumerate(q_values):
    q_signal[(t >= i) & (t < i + 1)] = val

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.2, 1.5)
axes[0].set_yticks([0, 1])

# Plot D
axes[1].fill_between(t, 0, d_signal, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, d_signal, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('D (input)', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.2, 1.5)
axes[1].set_yticks([0, 1])

# Plot Q
axes[2].fill_between(t, 0, q_signal, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, q_signal, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('Q (output)', fontsize=11, fontweight='bold')
axes[2].set_ylim(-0.2, 1.5)
axes[2].set_yticks([0, 1])
axes[2].set_xlabel('Clock Cycles', fontsize=11)

# Mark rising edges and show capture
for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#3b82f6', linestyle='-', lw=1, alpha=0.5)
    # Show capture arrows
    if i < 9:
        axes[0].annotate('', xy=(i+1, 0.5), xytext=(i+1, 1.3),
                        arrowprops=dict(arrowstyle='->', color='#ef4444', lw=1.5))

# Legend
axes[0].text(0.5, 1.35, '↓ Rising edges capture D into Q', fontsize=10, color='#ef4444')

plt.tight_layout()
plt.show()

**Key insight:** The Q output always reflects what D was *at the moment of the rising edge*. Changes to D between clock edges don't affect Q until the next rising edge.

---

## Setup and Hold Times

For reliable operation, data must be **stable** around the clock edge. This is defined by two critical timing parameters:

- **Setup time (t_su)**: Data must be stable *before* the clock edge
- **Hold time (t_h)**: Data must remain stable *after* the clock edge

Violating these requirements causes **metastability** — the flip-flop output becomes unpredictable.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(12, 5))
ax.set_title('Setup and Hold Time Requirements', fontsize=14, fontweight='bold')

t = np.arange(0, 4, 0.001)

# Clock edge at t=2
clk = np.zeros_like(t)
clk[t >= 2] = 1

# Data signal - stable around clock edge
data = np.zeros_like(t)
data[(t >= 0.5) & (t < 3.5)] = 1

# Plot with offset for visibility
ax.plot(t, clk * 0.8 + 1.5, color='#3b82f6', lw=2, drawstyle='steps-pre', label='CLK')
ax.plot(t, data * 0.8 + 0.3, color='#10b981', lw=2, drawstyle='steps-pre', label='D')

# Mark the clock edge
ax.axvline(x=2, color='#3b82f6', linestyle='-', lw=2, alpha=0.7)
ax.text(2, 2.6, 'Clock Edge', ha='center', fontsize=10, fontweight='bold', color='#3b82f6')

# Setup time region
ax.axvspan(1.3, 2, color='#fef3c7', alpha=0.7)
ax.annotate('', xy=(1.3, 0.1), xytext=(2, 0.1),
           arrowprops=dict(arrowstyle='<->', color='#f59e0b', lw=2))
ax.text(1.65, 0.0, 't_setup', ha='center', fontsize=11, fontweight='bold', color='#f59e0b')

# Hold time region
ax.axvspan(2, 2.5, color='#fee2e2', alpha=0.7)
ax.annotate('', xy=(2, 0.1), xytext=(2.5, 0.1),
           arrowprops=dict(arrowstyle='<->', color='#ef4444', lw=2))
ax.text(2.25, 0.0, 't_hold', ha='center', fontsize=11, fontweight='bold', color='#ef4444')

# Stable region annotation
ax.annotate('Data must be STABLE\nin this window', xy=(1.65, 0.7), xytext=(0.3, 0.5),
           fontsize=10, color='#374151',
           arrowprops=dict(arrowstyle='->', color='#374151', lw=1.5),
           bbox=dict(boxstyle='round', facecolor='white', edgecolor='#d1d5db'))

ax.set_ylim(-0.3, 2.8)
ax.set_xlim(0, 4)
ax.set_yticks([0.7, 1.9])
ax.set_yticklabels(['D', 'CLK'])
ax.set_xlabel('Time', fontsize=11)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

**Typical values** (vary by technology):

| Parameter | Typical Range | Notes |
|-----------|--------------|-------|
| Setup time | 0.1 - 2 ns | Data must arrive early enough |
| Hold time | 0.05 - 0.5 ns | Data must not change too quickly |

**Timing violations:**
- **Setup violation**: Data arrives too late → flip-flop may capture wrong value
- **Hold violation**: Data changes too early → flip-flop output becomes metastable

---

## Propagation Delay

Real circuits have **propagation delay** — the time for a signal change to travel through a component.

- **t_pd (propagation delay)**: Time from input change to output change
- **t_clk-to-q**: Time from clock edge to Q output change (for flip-flops)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(12, 6), sharex=True)
fig.suptitle('Propagation Delay in a D Flip-Flop', fontsize=14, fontweight='bold')

t = np.arange(0, 6, 0.001)
t_clk_to_q = 0.3  # Clock-to-Q delay

# Clock
clk = np.zeros_like(t)
for i in range(6):
    clk[(t >= i) & (t < i + 0.5)] = 1

# D input
d_signal = np.zeros_like(t)
d_signal[(t >= 0.6) & (t < 2.6)] = 1
d_signal[(t >= 3.6)] = 1

# Q output - delayed from clock edge by t_clk_to_q
q_signal = np.zeros_like(t)
q_signal[(t >= 1 + t_clk_to_q) & (t < 3 + t_clk_to_q)] = 1
q_signal[(t >= 4 + t_clk_to_q)] = 1

# Plot
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.2, 1.5)
axes[0].set_yticks([0, 1])

axes[1].fill_between(t, 0, d_signal, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, d_signal, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('D', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.2, 1.5)
axes[1].set_yticks([0, 1])

axes[2].fill_between(t, 0, q_signal, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, q_signal, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('Q', fontsize=11, fontweight='bold')
axes[2].set_ylim(-0.2, 1.5)
axes[2].set_yticks([0, 1])
axes[2].set_xlabel('Time', fontsize=11)

# Show t_clk_to_q delay
for edge in [1, 3, 4]:
    axes[0].axvline(x=edge, color='#3b82f6', linestyle='--', lw=1, alpha=0.7)
    axes[2].axvline(x=edge, color='#3b82f6', linestyle='--', lw=1, alpha=0.7)
    axes[2].axvline(x=edge + t_clk_to_q, color='#f59e0b', linestyle='--', lw=1, alpha=0.7)

# Annotate delay
axes[2].annotate('', xy=(1, 1.3), xytext=(1 + t_clk_to_q, 1.3),
                arrowprops=dict(arrowstyle='<->', color='#ef4444', lw=2))
axes[2].text(1 + t_clk_to_q/2, 1.4, 't_clk-to-q', ha='center', fontsize=10, color='#ef4444', fontweight='bold')

plt.tight_layout()
plt.show()

**Why propagation delay matters:**

1. **Limits maximum clock frequency**: Signals must propagate before the next clock edge
2. **Determines timing margins**: Delay reduces available setup time
3. **Causes timing paths**: Critical path = longest delay through combinational logic

---

## Example: Reading a Memory Interface

Let's examine a realistic timing diagram for a simple synchronous memory read operation:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(6, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Synchronous Memory Read Timing', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Chip Select (active low) - asserted at cycle 2
cs_n = np.ones_like(t)
cs_n[(t >= 2) & (t < 7)] = 0

# Read Enable (active low) - asserted at cycle 3
rd_n = np.ones_like(t)
rd_n[(t >= 3) & (t < 6)] = 0

# Address bus
addr_values = ['--', '--', '00', '00', '00', '00', '00', '--', '--', '--']

# Data bus - valid 1 cycle after read enable
data_values = ['ZZ', 'ZZ', 'ZZ', 'ZZ', '42', '42', 'ZZ', 'ZZ', 'ZZ', 'ZZ']

# Data Valid signal
valid = np.zeros_like(t)
valid[(t >= 4) & (t < 6)] = 1

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

# Plot CS_N
axes[1].fill_between(t, 0, cs_n, color='#ef4444', alpha=0.3, step='pre')
axes[1].plot(t, cs_n, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('CS_N', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])
axes[1].text(4.5, 0.2, 'chip selected', fontsize=9, ha='center', color='#ef4444')

# Plot RD_N
axes[2].fill_between(t, 0, rd_n, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, rd_n, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('RD_N', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].text(4.5, 0.2, 'read active', fontsize=9, ha='center', color='#f59e0b')

# Plot Address bus
axes[3].fill_between(t, 0.2, 0.8, color='#8b5cf6', alpha=0.2)
axes[3].plot(t, np.ones_like(t) * 0.8, color='#8b5cf6', lw=2)
axes[3].plot(t, np.ones_like(t) * 0.2, color='#8b5cf6', lw=2)
for i in range(1, 10):
    if addr_values[i] != addr_values[i-1]:
        axes[3].plot([i, i], [0.2, 0.8], color='#8b5cf6', lw=2)
        axes[3].plot([i-0.05, i+0.05], [0.2, 0.8], color='#8b5cf6', lw=1)
        axes[3].plot([i-0.05, i+0.05], [0.8, 0.2], color='#8b5cf6', lw=1)
for i, val in enumerate(addr_values):
    axes[3].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color='#5b21b6')
axes[3].set_ylabel('ADDR', fontsize=10, fontweight='bold')
axes[3].set_ylim(-0.1, 1.1)
axes[3].set_yticks([])

# Plot Data bus
axes[4].fill_between(t, 0.2, 0.8, color='#10b981', alpha=0.2)
axes[4].plot(t, np.ones_like(t) * 0.8, color='#10b981', lw=2)
axes[4].plot(t, np.ones_like(t) * 0.2, color='#10b981', lw=2)
for i in range(1, 10):
    if data_values[i] != data_values[i-1]:
        axes[4].plot([i, i], [0.2, 0.8], color='#10b981', lw=2)
        axes[4].plot([i-0.05, i+0.05], [0.2, 0.8], color='#10b981', lw=1)
        axes[4].plot([i-0.05, i+0.05], [0.8, 0.2], color='#10b981', lw=1)
for i, val in enumerate(data_values):
    color = '#065f46' if val not in ['ZZ', '--'] else '#9ca3af'
    axes[4].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color=color)
axes[4].set_ylabel('DATA', fontsize=10, fontweight='bold')
axes[4].set_ylim(-0.1, 1.1)
axes[4].set_yticks([])

# Plot Valid
axes[5].fill_between(t, 0, valid, color='#22c55e', alpha=0.3, step='pre')
axes[5].plot(t, valid, color='#22c55e', lw=2, drawstyle='steps-pre')
axes[5].set_ylabel('VALID', fontsize=10, fontweight='bold')
axes[5].set_ylim(-0.2, 1.4)
axes[5].set_yticks([0, 1])
axes[5].set_xlabel('Clock Cycles', fontsize=11)
axes[5].text(5, 1.2, 'Data can be sampled', fontsize=9, ha='center', color='#22c55e')

# Mark clock edges
for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

# Annotate phases
axes[0].text(2.5, 1.3, '1. Select chip', fontsize=9, ha='center', color='#374151')
axes[0].text(3.5, 1.3, '2. Assert read', fontsize=9, ha='center', color='#374151')
axes[0].text(5, 1.3, '3. Capture data', fontsize=9, ha='center', color='#374151')

plt.tight_layout()
plt.show()

**Reading this timing diagram:**

1. **Cycle 2**: Address is placed on bus, chip select asserted (CS_N goes low)
2. **Cycle 3**: Read enable asserted (RD_N goes low)
3. **Cycle 4**: Memory responds with data, VALID goes high
4. **Cycles 4-5**: Data is valid — controller samples it on the rising edge
5. **Cycle 6**: Read complete, signals deasserted

**Notation conventions:**
- **ZZ**: High-impedance (tri-state, bus not driven)
- **--**: Don't care (value irrelevant)
- **_N suffix**: Active-low signal

---

## Common Timing Diagram Patterns

### Pattern 1: Handshaking (Request/Acknowledge)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(12, 5), sharex=True)
fig.suptitle('Request/Acknowledge Handshaking', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Request
req = np.zeros_like(t)
req[(t >= 2) & (t < 6)] = 1

# Acknowledge (delayed response)
ack = np.zeros_like(t)
ack[(t >= 4) & (t < 7)] = 1

# Plot
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

axes[1].fill_between(t, 0, req, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, req, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('REQ', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])

axes[2].fill_between(t, 0, ack, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, ack, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('ACK', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].set_xlabel('Clock Cycles', fontsize=11)

# Annotations
axes[1].annotate('Request asserted', xy=(2.5, 1.1), fontsize=9, color='#10b981')
axes[2].annotate('Acknowledge returned', xy=(4.5, 1.1), fontsize=9, color='#f59e0b')
axes[1].annotate('Request deasserted\nafter ACK seen', xy=(6.5, 0.6), fontsize=9, color='#6b7280')

for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

### Pattern 2: Pipeline Stages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 1, figsize=(14, 7), sharex=True)
fig.suptitle('3-Stage Pipeline Timing', fontsize=14, fontweight='bold')

t = np.arange(0, 12, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(12):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Define colors for each instruction
colors = ['#3b82f6', '#10b981', '#f59e0b', '#ef4444', '#8b5cf6']
instr_names = ['A', 'B', 'C', 'D', 'E']

# Stage 1: Fetch
# Stage 2: Decode (1 cycle after fetch)
# Stage 3: Execute (2 cycles after fetch)

def draw_pipeline_stage(ax, stage_data, ylabel):
    ax.fill_between(t, 0.2, 0.8, color='#f3f4f6', alpha=0.5)
    ax.plot(t, np.ones_like(t) * 0.8, color='#9ca3af', lw=1)
    ax.plot(t, np.ones_like(t) * 0.2, color='#9ca3af', lw=1)
    
    for start, name, color in stage_data:
        ax.fill_between(t[(t >= start) & (t < start + 1)], 0.2, 0.8, color=color, alpha=0.6, step='pre')
        ax.text(start + 0.5, 0.5, name, ha='center', va='center', fontsize=11, fontweight='bold', color='white')
        ax.axvline(x=start, color='#374151', lw=1)
        ax.axvline(x=start+1, color='#374151', lw=1)
    
    ax.set_ylabel(ylabel, fontsize=10, fontweight='bold')
    ax.set_ylim(-0.1, 1.1)
    ax.set_yticks([])

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

# Fetch stage
fetch_data = [(1, 'A', colors[0]), (2, 'B', colors[1]), (3, 'C', colors[2]), 
              (4, 'D', colors[3]), (5, 'E', colors[4])]
draw_pipeline_stage(axes[1], fetch_data, 'FETCH')

# Decode stage (shifted by 1)
decode_data = [(2, 'A', colors[0]), (3, 'B', colors[1]), (4, 'C', colors[2]),
               (5, 'D', colors[3]), (6, 'E', colors[4])]
draw_pipeline_stage(axes[2], decode_data, 'DECODE')

# Execute stage (shifted by 2)
exec_data = [(3, 'A', colors[0]), (4, 'B', colors[1]), (5, 'C', colors[2]),
             (6, 'D', colors[3]), (7, 'E', colors[4])]
draw_pipeline_stage(axes[3], exec_data, 'EXECUTE')
axes[3].set_xlabel('Clock Cycles', fontsize=11)

# Mark clock edges
for i in range(12):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

# Annotations
axes[0].text(6.5, 1.3, 'At cycle 5: A executing, B decoding, C fetching', fontsize=10, color='#374151')

plt.tight_layout()
plt.show()

**Pipeline insight:** At any given cycle, multiple instructions are in flight simultaneously:
- Each stage processes a different instruction
- One instruction completes per cycle (after the pipeline fills)
- Latency = 3 cycles, but throughput = 1 instruction/cycle

---

## Timing Hazards and Issues

### Glitches

A **glitch** is an unwanted short pulse caused by unequal propagation delays through different logic paths:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 1, figsize=(12, 6), sharex=True)
fig.suptitle('Glitch in Combinational Logic (Y = A AND NOT B)', fontsize=14, fontweight='bold')

t = np.arange(0, 6, 0.001)

# A changes at t=2
a_signal = np.zeros_like(t)
a_signal[t >= 2] = 1

# B changes at t=2 (simultaneously in ideal world)
b_signal = np.zeros_like(t)
b_signal[t >= 2] = 1

# NOT B (with small delay of 0.1)
not_b = np.ones_like(t)
not_b[t >= 2.1] = 0

# Y = A AND NOT_B - shows glitch because A rises before NOT_B falls
y_signal = np.zeros_like(t)
y_signal[(t >= 2) & (t < 2.1)] = 1  # Glitch!

# Plot
axes[0].fill_between(t, 0, a_signal, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, a_signal, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('A', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

axes[1].fill_between(t, 0, b_signal, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, b_signal, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('B', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])

axes[2].fill_between(t, 0, not_b, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, not_b, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('NOT B', fontsize=11, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].annotate('Delayed!', xy=(2.1, 0.5), xytext=(2.5, 1.1),
                fontsize=9, color='#f59e0b',
                arrowprops=dict(arrowstyle='->', color='#f59e0b'))

axes[3].fill_between(t, 0, y_signal, color='#ef4444', alpha=0.3, step='pre')
axes[3].plot(t, y_signal, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[3].set_ylabel('Y', fontsize=11, fontweight='bold')
axes[3].set_ylim(-0.2, 1.4)
axes[3].set_yticks([0, 1])
axes[3].set_xlabel('Time', fontsize=11)

# Highlight glitch
axes[3].axvspan(2, 2.1, color='#fecaca', alpha=0.7)
axes[3].annotate('GLITCH!', xy=(2.05, 1.0), xytext=(2.5, 1.2),
                fontsize=10, fontweight='bold', color='#ef4444',
                arrowprops=dict(arrowstyle='->', color='#ef4444', lw=2))

axes[0].axvline(x=2, color='#d1d5db', linestyle='--', lw=1)

plt.tight_layout()
plt.show()

**Why glitches occur:** When A and B both change from 0→1:
- Ideally: Y stays 0 (since A·B̄ = 1·0 = 0)
- Reality: A rises immediately, but NOT B takes time to fall
- Brief moment: A=1, NOT B=1 (hasn't updated yet) → Y=1 (glitch!)

**Avoiding glitch problems:**
- Register outputs to sample only at clock edges
- Use synchronous design practices
- Add glitch filters for asynchronous inputs

---

## Key Takeaways

1. **Timing diagrams are essential** for understanding and debugging digital circuits

2. **Clock edges are reference points** — in synchronous design, everything happens relative to the clock

3. **Setup and hold times** define when data must be stable for reliable capture

4. **Propagation delay** limits how fast your circuit can run

5. **Common notation:**
   - Active-low signals: _N suffix or overbar
   - High-impedance: Z or Hi-Z
   - Don't care: X or --
   - Bus transitions: X-crossing pattern

6. **Practice reading timing diagrams** from datasheets — they're the universal language of digital hardware

---

*Timing diagrams bridge the gap between abstract logic design and physical hardware reality. Master them, and you'll be able to debug the most challenging timing issues.*